In [70]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 



# Read The Data

In [37]:
df = pd.read_csv('Data\BreasetConser.csv')

**shows the first 5 columns**  

In [38]:
df.head(5)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-05-20,2018-08-26,Alive
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-04-26,2019-01-25,Dead
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,2018-08-24,2020-04-08,Alive
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-11-16,2020-07-28,Alive
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,2018-12-12,2019-01-05,Alive


**Preparing X and Y** 

**Categories columns**

In [39]:
print("Categories in 'gender' variable:  ",end=" ")
print(df['Gender'].unique())
print("Categories in 'Tumor stage' variable:  ",end=" ")
print(df['Tumour_Stage'].unique())
print("Categories in 'Histology' variable:  ",end=" ")
print(df['Histology'].unique())
print("Categories in 'ER status' variable:  ",end=" ")
print(df['ER status'].unique())
print("Categories in 'PR status' variable:  ",end=" ")
print(df['PR status'].unique())
print("Categories in 'HER2 status' variable:  ",end=" ")
print(df['HER2 status'].unique())
print("Categories in 'Surgery Type' variable:  ",end=" ")
print(df['Surgery_type'].unique())

Categories in 'gender' variable:   ['FEMALE' 'MALE']
Categories in 'Tumor stage' variable:   ['II' 'I' 'III']
Categories in 'Histology' variable:   ['Infiltrating Ductal Carcinoma' 'Infiltrating Lobular Carcinoma'
 'Mucinous Carcinoma']
Categories in 'ER status' variable:   ['Positive']
Categories in 'PR status' variable:   ['Positive']
Categories in 'HER2 status' variable:   ['Negative' 'Positive']
Categories in 'Surgery Type' variable:   ['Other' 'Lumpectomy' 'Modified Radical Mastectomy' 'Simple Mastectomy']


In [40]:
df['Date_of_Last_Visit']=pd.to_datetime(df['Date_of_Last_Visit'])
df['Date_of_Surgery']= pd.to_datetime(df['Date_of_Surgery'])

In [41]:
#extract day,monthm,year from Date_of_Last_Visit
df['year_Visit'] = df['Date_of_Last_Visit'].dt.year
df['month_Visit'] = df['Date_of_Last_Visit'].dt.month
df['day_Visit'] = df['Date_of_Last_Visit'].dt.day
df.drop(columns='Date_of_Last_Visit',axis=1,inplace=True)

#extract day,month,year from Date_of_Last_Visit
df['year_Surgery'] = df['Date_of_Surgery'].dt.year
df['month_Surgery'] = df['Date_of_Surgery'].dt.month
df['day_Surgery'] = df['Date_of_Surgery'].dt.day
df.drop(columns='Date_of_Surgery',axis=1,inplace=True)



In [42]:
df.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Patient_Status,year_Visit,month_Visit,day_Visit,year_Surgery,month_Surgery,day_Surgery
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive,2018,8,26,2018,5,20
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Dead,2019,1,25,2018,4,26
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,Alive,2020,4,8,2018,8,24
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive,2020,7,28,2018,11,16
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,Alive,2019,1,5,2018,12,12


In [43]:
X = df.drop(columns='Patient_Status',axis=1)
X.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,year_Visit,month_Visit,day_Visit,year_Surgery,month_Surgery,day_Surgery
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018,8,26,2018,5,20
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2019,1,25,2018,4,26
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,2020,4,8,2018,8,24
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2020,7,28,2018,11,16
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,2019,1,5,2018,12,12


In [55]:
y= df['Patient_Status']
y.sample(7)

19     Alive
35     Alive
68     Alive
192     Dead
142    Alive
126    Alive
94      Dead
Name: Patient_Status, dtype: object

In [68]:
#Create Columns with 3 types of of transformers 

catgorical_features = X.select_dtypes(include='O')
catgorical_features  = catgorical_features.drop(columns='Gender',axis = 1)
catgorical_features 

,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type
0,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other
1,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other
2,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy
3,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other
4,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy
...,...,...,...,...,...,...
312,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy
313,I,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Modified Radical Mastectomy
314,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Simple Mastectomy
315,II,Infiltrating Lobular Carcinoma,Positive,Positive,Negative,Lumpectomy


In [71]:
Gender_feature = X['Gender']
Gender_feature

0      FEMALE
1      FEMALE
2      FEMALE
3      FEMALE
4      FEMALE
        ...  
312    FEMALE
313    FEMALE
314    FEMALE
315    FEMALE
316    FEMALE
Name: Gender, Length: 317, dtype: object

In [64]:
numerical_features = X.select_dtypes(exclude='O')
numerical_features

,Age,Protein1,Protein2,Protein3,Protein4,year_Visit,month_Visit,day_Visit,year_Surgery,month_Surgery,day_Surgery
0,42,0.952560,2.15000,0.007972,-0.048340,2018,8,26,2018,5,20
1,54,0.000000,1.38020,-0.498030,-0.507320,2019,1,25,2018,4,26
2,63,-0.523030,1.76400,-0.370190,0.010815,2020,4,8,2018,8,24
3,78,-0.876180,0.12943,-0.370380,0.132190,2020,7,28,2018,11,16
4,42,0.226110,1.74910,-0.543970,-0.390210,2019,1,5,2018,12,12
...,...,...,...,...,...,...,...,...,...,...,...
312,59,0.024598,1.40050,0.024751,0.280320,2020,3,27,2019,1,15
313,41,0.100120,-0.46547,0.472370,-0.523870,2019,4,23,2018,7,25
314,54,0.753820,1.64250,-0.332850,0.857860,2019,10,11,2019,3,26
315,74,0.972510,1.42680,-0.366570,-0.107820,2018,12,5,2018,11,26


In [65]:
X.shape

(317, 18)

In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer

   
